Importações

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from pt_lemmatizer import Lemmatizer
import spacy
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 42
nlp = spacy.load("pt_core_news_sm")


Função de Detetcção de Linguagens utilizando langdetect

In [3]:

def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"

Funções de Tratamento de Textos para Tweets

In [4]:
punctuation ="‘!”$%&\’()*+,-./:;<=>?[\\]^_`{|}~•@’"


def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
    tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove tweeted at
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

def remove_risadas(tweet):
    tweet = re.sub(r'\b[kK]{2,}\b', '', tweet)  # kkk, kkkkk...
    tweet = re.sub(r'\b(?:[hH][aA]){2,}\b', '', tweet)  # haha, hahahaha...
    tweet = re.sub(r'\b(?:[jJ][aA]){2,}\b', '', tweet)  # jaja, jajajaja... (mínimo 2x)
    tweet = re.sub(r'\b(?:[rR][sS]){2,}\b', '', tweet)  # rsrs, rsrsrsrs...
    tweet = re.sub(r'\b(?:[kK][sS]){2,}\b', '', tweet)  # ksks, ksksksks...
    return tweet

import re

def remove_emojis(tweet):
    """Remove emojis e caracteres especiais não ASCII comuns em emotes"""
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons 😀-🙏
        "\U0001F300-\U0001F5FF"  # símbolos e pictogramas 🌧-🗿
        "\U0001F680-\U0001F6FF"  # transporte e mapas 🚀-🚧
        "\U0001F1E0-\U0001F1FF"  # bandeiras 🇧🇷-🇺🇸
        "\U00002700-\U000027BF"  # símbolos diversos ✂-➿
        "\U0001F900-\U0001F9FF"  # emojis suplementares 🤖-🧠
        "\U0001FA70-\U0001FAFF"  # objetos adicionais 🩰-🪅
        "\U00002500-\U00002BEF"  # caixas, setas, ideogramas
        "\U0001F018-\U0001F270"  # vários outros
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', tweet)



<>:14: SyntaxWarning: invalid escape sequence '\s'
<>:14: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Pessoal\AppData\Local\Temp\ipykernel_14952\3105653607.py:14: SyntaxWarning: invalid escape sequence '\s'
  tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet


Usamos os Spacy para dividir em Tokens

In [5]:
def tokenize(texto):
    """Tokeniza, remove stopwords e lematiza um texto em português"""
    
    custom_stopwords = {"kkk", "bom dia", "pra", "dia", "de"}
    for word in custom_stopwords:
        nlp.vocab[word].is_stop = True

    doc = nlp(texto)
    return [
        token.lemma_.lower()
        for token in doc
        if not token.is_stop and token.is_alpha and len(token.text) > 2
    ]

Exemplo de Uso

In [6]:
print(tokenize("Os ratos roeram a roupa do rei de Roma."))
# Saída exemplo: ['rato', 'roer', 'roupa', 'rei', 'Roma']

['rato', 'roer', 'roupa', 'rei', 'roma']


Funções de PreProcessamento de Tweets

In [7]:
def preprocess_tweet(tweet):
    """Main master function to clean tweets, stripping noisy characters, and tokenizing use lemmatization"""
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = remove_risadas(tweet)
    tweet = remove_emojis(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet_token_list = tokenize(tweet)  # apply lemmatization and tokenization
    tweet = ' '.join(tweet_token_list)
    return tweet

def basic_clean(tweet):
    """Main master function to clean tweets only without tokenization or removal of stopwords"""
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = remove_risadas(tweet)
    tweet = remove_emojis(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet = re.sub('📝 …', '', tweet)
    return tweet

<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:27: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:27: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Pessoal\AppData\Local\Temp\ipykernel_14952\503958352.py:11: SyntaxWarning: invalid escape sequence '\s'
  tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
C:\Users\Pessoal\AppData\Local\Temp\ipykernel_14952\503958352.py:27: SyntaxWarning: invalid escape sequence '\s'
  tweet = re.sub('\s+', ' ', tweet)  # remove double spacing


Aplicação no Dataframe de Tweets

In [8]:
def clean_tweets(df):
    """Main function to read in and return cleaned and preprocessed dataframe.
    """

    df['clean_tweet'] = df.Content.apply(basic_clean)
    num_tweets = len(df)
    print('Complete. Number of Tweets that have been cleaned : {}'.format(num_tweets))
    return df



def tokenize_tweets(df):
    """Main function to read in and return cleaned and preprocessed dataframe.
    This can be used in Jupyter notebooks by importing this module and calling the tokenize_tweets() function
    Args:
        df = data frame object to apply cleaning to
    Returns:
        pandas data frame with cleaned tokens
    """

    df['tokens'] = df.Content.apply(preprocess_tweet)
    num_tweets = len(df)
    print('Complete. Number of Tweets that have been tokenized : {}'.format(num_tweets))
    return df


def tokenize_news(df):
    """Main function to read in and return cleaned and preprocessed dataframe.
    This can be used in Jupyter notebooks by importing this module and calling the tokenize_tweets() function
    Args:
        df = data frame object to apply cleaning to
    Returns:
        pandas data frame with cleaned tokens
    """

    df['tokens'] = df.Corpo.apply(preprocess_tweet)
    num_tweets = len(df)
    print('Complete. Number of Tweets that have been tokenized : {}'.format(num_tweets))
    return df

def tokenize_gov(df):
    """Main function to read in and return cleaned and preprocessed dataframe.
    This can be used in Jupyter notebooks by importing this module and calling the tokenize_tweets() function
    Args:
        df = data frame object to apply cleaning to
    Returns:
        pandas data frame with cleaned tokens
    """

    df['tokens'] = df.Conteudo_Materia.apply(preprocess_tweet)
    num_tweets = len(df)
    print('Complete. Number of Tweets that have been tokenized : {}'.format(num_tweets))
    return df

# Esse trecho se dedica a Tratar os dados de Tweets

### Carregar a planilha e selecionar colunas relevantes

In [112]:
caminho_arquivo = r'C:\Users\Pessoal\Documents\Unifesp\Aries\aries_topic_selector\dados\dadosTwitter\CSV\tweets tuberculose.csv'
tweets_df = pd.read_csv(caminho_arquivo)
#tweets_df.dropna(axis='columns', inplace=True)
print(tweets_df.columns)

#Escolhendo as colunas relevantes
tweets_df = tweets_df[['Tweet ID', 'Content', 'Date']]


#Só por precaução, removendo duplicatas
tweets_df.drop_duplicates(inplace=True, subset="Tweet ID")
tweets_df.drop_duplicates(inplace=True, subset="Content")

print(tweets_df)


Index(['Tweet ID', 'URL', 'Content', 'Likes', 'Retweets', 'Replies', 'Quotes',
       'Views', 'Date'],
      dtype='object')
                 Tweet ID                                            Content  \
0     1932527926723182644            I hate Tuberculose with all of my heart   
1     1932521806503858629  Je suis pas préparée à voir la Tuberculose en ...   
2     1932513994868187138  vei esse módulo de ecla n faz sentido nenhum u...   
3     1932507835130368147  @weedtonina @livbadbiatch @ogmtbmr Bgl é sério...   
4     1932501301222650086  Je vais trouver la personne qui m’a mis cas co...   
...                   ...                                                ...   
7882  1889391439034876258  @MarionMarechal @ecrgroup @LaurenceTrochu @G_P...   
7883  1889391168334516509  Tuberculose-behandelprogramma's stopzetten is ...   
7884  1889391125984686508  @MarionMarechal je me suis mordre par le sopor...   
7885  1889388371497152671  @ansecivoc Tja, een beetje van alles plus tube.

Agora, separamos os tweets por Idiomas utilizando a Biblioteca LangDetect

In [113]:
tweets_df = tweets_df.dropna(subset=["Content"])
tweets_df = clean_tweets(tweets_df)
tweets_df["language"] = tweets_df["Content"].apply(detect_language)

#Criando dataframes separados por idioma

df_espanhol = tweets_df[tweets_df["language"] == "es"]
df_ingles = tweets_df[tweets_df["language"] == "en"]
df_ingles = tweets_df[tweets_df["language"] == "fr"]
df_portugues = tweets_df[~tweets_df["language"].isin(["es", "en", "fr"])]

df_espanhol.to_excel("tweets_processados_espanhol.xlsx", index=False)
df_ingles.to_excel("tweets_processados_ingles.xlsx", index=False)
df_portugues.to_excel("tweets_processados_portugues.xlsx", index=False)


Complete. Number of Tweets that have been cleaned : 7838


Agora, aplicas o Tokenizer no datframe em Portugues, que é o nosso foco

In [114]:

df_portugues = tokenize_tweets(df_portugues)

#Removendo tweets com menos de 6 palavras
df_portugues["word_count"] = df_portugues["tokens"].apply(lambda x: len(x.split()))
df_portugues = df_portugues[df_portugues["word_count"] >= 6]
df_portugues = df_portugues.drop(columns=["word_count"])
df_portugues.drop_duplicates(inplace=True, subset="tokens")
df_portugues.to_csv("tweets_processados.csv", index=False)
print("Arquivo 'tweets_processados.csv' salvo com sucesso!")

Complete. Number of Tweets that have been tokenized : 5074
Arquivo 'tweets_processados.csv' salvo com sucesso!


C:\Users\Pessoal\AppData\Local\Temp\ipykernel_10228\38493525.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df.Content.apply(preprocess_tweet)
C:\Users\Pessoal\AppData\Local\Temp\ipykernel_10228\4190352179.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_portugues["word_count"] = df_portugues["tokens"].apply(lambda x: len(x.split()))


Se quiser salvar como Xlsx

In [115]:
df_portugues.to_excel("tweets_processados.xlsx", index=False)
print("Arquivo 'tweets_processados.xlsx' salvo com sucesso!")

Arquivo 'tweets_processados.xlsx' salvo com sucesso!


# Esse trecho se dedica a tratar os trechos para Notícias do G1

### Carregar planilha e selecionar colunas relevantes

In [ ]:
caminho_arquivo = r'C:\Users\Pessoal\Documents\Unifesp\Aries\aries_topic_selector\dados\dadosG1\unificado\unificado.csv'
news_df = pd.read_csv(caminho_arquivo, delimiter=',')

print(news_df.columns)
print(list(news_df.columns))


#Escolhendo as colunas relevantes
news_df = news_df[['Titulo', 'Corpo']]


#Só por precaução, removendo duplicatas
news_df.drop_duplicates(inplace=True, subset="Titulo")
news_df.drop_duplicates(inplace=True, subset="Corpo")

print(news_df)

Index(['Titulo', 'Corpo', 'Link', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5',
       'Unnamed: 6'],
      dtype='object')
['Titulo', 'Corpo', 'Link', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6']
                                                 Titulo  \
0     Anvisa suspende amoxicilina e rifamicina da fa...   
1     Lotes de amoxicilina são proibidos de serem co...   
2                                 Título não encontrado   
7     Anvisa suspende por 90 dias venda de antibióti...   
8     Remédios e vacinas são encontrados no meio de ...   
...                                                 ...   
4354  Tuberculose no Pará foi tema de reunião com o ...   
4355  Hospital define local de exames após casos de ...   
4357  Número de pessoas com tuberculose recua pela 1...   
4358  Casos de tuberculose no mundo caem pela primei...   
4361  Número de crianças com tuberculose em Campinas...   

                                                  Corpo  
0     04/02/2015 20h31- Atual

Agora limpamos o dataFrame e usamos o tokenizer

In [12]:
# Suponha que seu DataFrame se chame `df` e a coluna com o texto seja "Corpo"
news_df = news_df.dropna(subset=["Corpo"])
news_df['Corpo'] = news_df['Corpo'].str.replace(
    r'^\d{2}/\d{2}/\d{4}\s\d{2}h\d{2}-\s*Atualizado em\d{2}/\d{2}/\d{4}\s\d{2}h\d{2}\s*,?\s*',
    '',
    regex=True
)
news_df = tokenize_news(news_df)

#Removendo tweets com menos de 6 palavras
news_df["word_count"] = news_df["tokens"].apply(lambda x: len(x.split()))
news_df = news_df[news_df["word_count"] >= 6]
news_df = news_df.drop(columns=["word_count"])
news_df.drop_duplicates(inplace=True, subset="tokens")
news_df.to_csv("news_processados.csv", index=False)



print("Arquivo 'noticias_g1_processados.csv' salvo com sucesso!")

Complete. Number of Tweets that have been tokenized : 2002
Arquivo 'noticias_g1_processados.csv' salvo com sucesso!


In [13]:
news_df.to_excel("news_processados.xlsx", index=False)
print("Arquivo 'news_processados.xlsx' salvo com sucesso!")

Arquivo 'news_processados.xlsx' salvo com sucesso!


In [14]:


news_df

,Titulo,Corpo,tokens
0,Anvisa suspende amoxicilina e rifamicina da fa...,"Do G1, em São Paulo A Anvisa anunciou nesta qu...",paulo anvisa anunciar feira suspensão antibiót...
1,Lotes de amoxicilina são proibidos de serem co...,Do RJTV Conteúdo não disponível. Infelizmente ...,rjtv conteúdo disponível infelizmente vídeo di...
2,Título não encontrado,"Por Kleber Tomaz, G1 SP— São Paulo 05/11/2019 ...",kleber tomaz paulo hatualizadohá ano antibióti...
7,Anvisa suspende por 90 dias venda de antibióti...,"Do G1, em São Paulo A Agência Nacional de Vigi...",paulo agência nacional vigilância sanitária an...
8,Remédios e vacinas são encontrados no meio de ...,Do G1 Ribeirão e Franca Jovens entre 6 e 17 a...,ribeirão franca jovem ano aprender instrumento...
...,...,...,...
4354,Tuberculose no Pará foi tema de reunião com o ...,Do G1 PA A situação da tuberculose no Pará foi...,situação tuberculose par tema reunião secretar...
4355,Hospital define local de exames após casos de ...,Do G1 Campinas e Região O Hospital e Maternid...,campina região hospital maternidade madre theo...
4357,Número de pessoas com tuberculose recua pela 1...,"Do G1, com informações da Reuters O número de ...",informação reuter pessoa doente conta tubercul...
4358,Casos de tuberculose no mundo caem pela primei...,Reuters (Reuters) - O número de pessoas que ad...,reuter reuters pessoa adoecer tuberculose cair...


Se quiser salvar como Xlsx

# Esse trecho se dedica a tratar os trechos para Notícias do GOV

### Carregar planilha e selecionar colunas relevantes

In [193]:
caminho_arquivo = r'C:\Users\Pessoal\Documents\Unifesp\Aries\aries_topic_selector\dados\dadosGov\unificado\unificado.csv'
news_df = pd.read_csv(caminho_arquivo, delimiter=',', quotechar='"')
print(news_df.columns)
print(list(news_df.columns))


#Escolhendo as colunas relevantes
news_df = news_df[['Titulo', 'Conteudo_Materia']]


#Só por precaução, removendo duplicatas
news_df.drop_duplicates(inplace=True, subset="Titulo")
news_df.drop_duplicates(inplace=True, subset="Conteudo_Materia")

print(news_df)

Index(['Titulo', 'Descricao', 'Link', 'Conteudo_Materia'], dtype='object')
['Titulo', 'Descricao', 'Link', 'Conteudo_Materia']
                                                 Titulo  \
0     Comunidades indígenas do Vale do Javari ganham...   
1     Mutirão leva atendimento médico e insumos ao V...   
2     Governo brasileiro envia insumos e medicamento...   
3     Reajuste de medicamentos é o menor registrado ...   
4     DSEI Alto Rio Juruá recebe medicamentos da Ren...   
...                                                 ...   
1497  DSEI Kaiapó do Pará articula parceria com hosp...   
1499  DSEI Médio Rio Solimões realiza ação de saúde ...   
1501  DSEI Vilhena promove capacitação para profissi...   
1503  DSEI Parintins participa de mostra em parceria...   
1505  DSEI Kayapó Mato Grosso realiza ação sobre cui...   

                                       Conteudo_Materia  
0     Comunidades da Terra Indígena do Vale do Javar...  
1     O Governo Federal vai realizar um novo mut

In [194]:
news_df = news_df.dropna(subset=["Conteudo_Materia"])
news_df = tokenize_gov(news_df)

#Removendo tweets com menos de 6 palavras
news_df["word_count"] = news_df["tokens"].apply(lambda x: len(x.split()))
news_df = news_df[news_df["word_count"] >= 6]
news_df = news_df.drop(columns=["word_count"])
news_df.drop_duplicates(inplace=True, subset="tokens")
news_df.to_csv("news_processados.csv", index=False)



print("Arquivo 'noticias_g1_processados.csv' salvo com sucesso!")

Complete. Number of Tweets that have been tokenized : 1223
Arquivo 'noticias_g1_processados.csv' salvo com sucesso!


In [195]:
news_df.to_excel("news_processados.xlsx", index=False)
print("Arquivo 'news_processados.xlsx' salvo com sucesso!")

Arquivo 'news_processados.xlsx' salvo com sucesso!


In [180]:
news_df

,Titulo,Conteudo_Materia,tokens
0,Pesquisa inédita financiada pelo Ministério da...,O Ministério da Saúde financiou um estudo inéd...,ministério saúde financiar estudo inédito prog...
1,PPSUS em Roraima terá pesquisas voltadas para ...,"O Ministério da Saúde, por meio do Departament...",ministério saúde departamento ciência tecnolog...
2,Brasil amplia prevenção e aposta em tratamento...,O Brasil vem intensificando os esforços para e...,brasil intensificar esforço eliminar atubercul...
3,Brasil firma acordo com São Tomé e Príncipe pa...,Brasil e São Tomé e Príncipe vão desenvolver u...,brasil tomé príncipe desenvolver amplo coopera...
4,Saúde lança painel de doenças e agravos na pop...,OCentro Nacional de Inteligência Epidemiológic...,ocentro nacional inteligência epidemiológico v...
...,...,...,...
781,DSEI Kaiapó do Pará articula parceria com hosp...,"Na última semana, o Distrito Sanitário Especia...",último semana distrito sanitário especial indí...
783,DSEI Médio Rio Solimões realiza ação de saúde ...,As equipes da Divisão de Atenção à Saúde Indíg...,equipe divisão atenção saúde indígeno diasi se...
785,DSEI Vilhena promove capacitação para profissi...,Termina nesta quinta-feira (29) a capacitação ...,terminar feira capacitação emtuberculose hanse...
787,DSEI Parintins participa de mostra em parceria...,O Distrito Sanitário Especial Indígena (DSEI) ...,distrito sanitário especial indígena dsar pari...
